In [2]:
import tensorflow as tf
from keras import models, layers, optimizers
from keras.layers import Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import ResNet50
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
IMAGE_SIZE = 224
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS=25

In [ ]:
base_model = ResNet50(
    include_top = False,
    input_shape = (224,224,3),
    weights = 'imagenet'
)

94765736/94765736 [==============================] - 1s 0us/step


In [ ]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# base_model.summary()

# for layer in base_model.layers:
#     print(layer.name, layer.trainable)

for i, layer in enumerate(base_model.layers):
   print(i, layer.name, layer.trainable)

0 input_1 False
1 conv1_pad False
2 conv1_conv False
3 conv1_bn False
4 conv1_relu False
5 pool1_pad False
6 pool1_pool False
7 conv2_block1_1_conv False
8 conv2_block1_1_bn False
9 conv2_block1_1_relu False
10 conv2_block1_2_conv False
11 conv2_block1_2_bn False
12 conv2_block1_2_relu False
13 conv2_block1_0_conv False
14 conv2_block1_3_conv False
15 conv2_block1_0_bn False
16 conv2_block1_3_bn False
17 conv2_block1_add False
18 conv2_block1_out False
19 conv2_block2_1_conv False
20 conv2_block2_1_bn False
21 conv2_block2_1_relu False
22 conv2_block2_2_conv False
23 conv2_block2_2_bn False
24 conv2_block2_2_relu False
25 conv2_block2_3_conv False
26 conv2_block2_3_bn False
27 conv2_block2_add False
28 conv2_block2_out False
29 conv2_block3_1_conv False
30 conv2_block3_1_bn False
31 conv2_block3_1_relu False
32 conv2_block3_2_conv False
33 conv2_block3_2_bn False
34 conv2_block3_2_relu False
35 conv2_block3_3_conv False
36 conv2_block3_3_bn False
37 conv2_block3_add False
38 conv2_bloc

In [ ]:
# for layer in base_model.layers[165:]:
#    layer.trainable = True

# for i, layer in enumerate(base_model.layers):
#    print(i, layer.name, layer.trainable)

In [ ]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [ ]:
model = models.Sequential()

model.add(base_model)
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 64)                6422592   
                                                                 
 dense_1 (Dense)             (None, 128)               8320      
                                                                 
 dense_2 (Dense)             (None, 2)                 258       
                                                                 
Total params: 30018882 (114.51 MB)
Trainable params: 6431170 (24.53 MB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = datagen.flow_from_directory(
    r"/content/drive/MyDrive/chest_x_ray_aug/train",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        r"/content/drive/MyDrive/chest_x_ray_aug/valid",
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical')

Found 12230 images belonging to 2 classes.
Found 1747 images belonging to 2 classes.


In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator
)

Epoch 1/25
383/383 [==============================] - 2177s 6s/step - loss: 0.5716 - accuracy: 0.7545 - val_loss: 0.3679 - val_accuracy: 0.8472
Epoch 2/25
383/383 [==============================] - 207s 540ms/step - loss: 0.4101 - accuracy: 0.8019 - val_loss: 0.3420 - val_accuracy: 0.8460
Epoch 3/25
383/383 [==============================] - 207s 540ms/step - loss: 0.3896 - accuracy: 0.8141 - val_loss: 0.3318 - val_accuracy: 0.8598
Epoch 4/25
383/383 [==============================] - 208s 543ms/step - loss: 0.3694 - accuracy: 0.8257 - val_loss: 0.3032 - val_accuracy: 0.8580
Epoch 5/25
383/383 [==============================] - 210s 548ms/step - loss: 0.3557 - accuracy: 0.8358 - val_loss: 0.3038 - val_accuracy: 0.8683
Epoch 6/25
383/383 [==============================] - 209s 545ms/step - loss: 0.3646 - accuracy: 0.8325 - val_loss: 0.3081 - val_accuracy: 0.8655
Epoch 7/25
383/383 [==============================] - 204s 534ms/step - loss: 0.3695 - accuracy: 0.8252 - val_loss: 0.3245 - v

In [ ]:
model.save(r"/content/drive/MyDrive/models_ResNet50/1")

In [ ]:
from keras import models, layers
MODEL = tf.keras.models.load_model(r"/content/drive/MyDrive/models_ResNet50/1")

In [4]:
# from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
# import numpy as np
# # Get true labels and predicted labels for validation data
# val_data, val_labels = validation_generator.next()
# val_pred_probs = MODEL.predict(val_data)
# val_pred_labels = np.argmax(val_pred_probs, axis=1)  # Convert predicted probabilities to labels

# # Calculate accuracy
# accuracy = accuracy_score(np.argmax(val_labels, axis=1), val_pred_labels)

# # Calculate precision
# precision = precision_score(np.argmax(val_labels, axis=1), val_pred_labels)

# # Calculate recall
# recall = recall_score(np.argmax(val_labels, axis=1), val_pred_labels)

# print("Accuracy:", accuracy)
# print("Precision:", precision)
# print("Recall:", recall)

# # You can also print a classification report which includes precision, recall, and F1-score for each class
# print(classification_report(np.argmax(val_labels, axis=1), val_pred_labels))



import tensorflow as tf
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
import numpy as np

# Load the pre-trained model
model = tf.keras.models.load_model(r"D:\Deep Learning & ML\pneumonia_detection\Project_zip\Models\models_ResNet50\1")

# Generate predictions on the validation data
test_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = test_datagen.flow_from_directory(
    r"D:\Deep Learning & ML\pneumonia_detection\training\chest_x_ray_aug\valid",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False  # Important: do not shuffle for evaluation
)

# Get true labels and predicted labels
Y_true = validation_generator.classes
Y_pred_prob = model.predict(validation_generator)
Y_pred = np.argmax(Y_pred_prob, axis=1)

# Compute precision, recall, and F1 score
precision = precision_score(Y_true, Y_pred)
recall = recall_score(Y_true, Y_pred)
f1 = f1_score(Y_true, Y_pred)

# Print the results
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Print detailed classification report
print(classification_report(Y_true, Y_pred, target_names=validation_generator.class_indices.keys()))

Found 1747 images belonging to 2 classes.
55/55 [==============================] - 131s 2s/step
Precision: 0.9351
Recall: 0.9182
F1 Score: 0.9266
              precision    recall  f1-score   support

      NORMAL       0.79      0.83      0.81       475
   PNEUMONIA       0.94      0.92      0.93      1272

    accuracy                           0.89      1747
   macro avg       0.86      0.87      0.87      1747
weighted avg       0.90      0.89      0.89      1747



In [4]:
from email.mime import image
from pyexpat import model
import tensorflow as tf
from keras import models, layers
import matplotlib.pyplot as plt
import cv2
import numpy as np

CLASS_NAMES = ['NORMAL', 'PNEUMONIA']

MODEL = tf.keras.models.load_model(r"/content/drive/MyDrive/models_ResNet50/1")

image = cv2.imread(r"/content/drive/MyDrive/chest_x_ray_aug/train/PNEUMONIA/download.jpg")

# print(image.shape)

resized_image = cv2.resize(image, (224, 224))

# print(resized_image.shape)

img_array = tf.keras.preprocessing.image.img_to_array(resized_image)
img_array = tf.expand_dims(img_array, 0) # Create a batch
img_array = img_array / 255.0

predictions = MODEL.predict(img_array)


predicted_class = CLASS_NAMES[np.argmax(predictions[0])]
# confidence = np.max(predictions[0])
confidence = round(100 * (np.max(predictions[0])), 2)

print(f"Prediction: {predicted_class} \nAccurecy: {confidence}%")


1/1 [==============================] - 1s 880ms/step
Prediction: NORMAL 
Accurecy: 58.39%
